In [ ]:
# [Modified] 改用 Finlab 資料進行分析 (包含分時交易資訊)
# [Modified] 改用 Finlab 資料進行抓取 (支援處置開始時間)
import pandas as pd
from tqdm import tqdm
from loguru import logger
import sys
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2
sys.path.append("/Users/xinc./Documents/GitHub/note")
sys.path.append(os.getcwd()) # 加入目前路徑以匯入 utils

from module.get_info_Postgre import PostgresClient
from module.get_info_FinMind import FinMindClient
from module.plot_func import plot
from utils import batch_fetch_prices, run_event_study

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# read

### postgresql

In [2]:
# 讀取資料庫中的 disposal_info 表格
database_name = "disposal"
client = PostgresClient(database = database_name)
disposal_info = client.fetch_table(table ='disposal_info', date_col='date', start_date='2018-01-01', end_date='2025-12-31', order_by='date')
disposal_info = disposal_info.loc[disposal_info["is_disposal"] == True]

### finmind

In [ ]:
# [Optimization] 使用平行化抓取加速下載
# 關閉 Log 避免洗版
logger.remove()
logger.add(sys.stderr, level="WARNING")

fm_client = FinMindClient()
price_df = batch_fetch_prices(fm_client, disposal_info, max_workers=10)

if not price_df.empty:
    print(price_df.head())
else:
    print("No price data fetched.")

gc.collect()

Found 895 unique stocks with disposal events.


Fetching by Stock: 100%|██████████| 895/895 [02:50<00:00,  5.25it/s]

Fetched 33450 price rows.
        Date Stock_id   Open   High    Low  Close  Volume  TradingAmount
0 2017-12-29     1475  48.00  53.30  48.00  50.00  901560       45481144
1 2018-01-02     1475  48.20  48.20  45.00  45.00  169974        7779259
2 2018-01-03     1475  45.10  48.00  45.10  45.85   87200        4039709
3 2018-01-04     1475  46.95  47.95  46.00  46.80   65042        3053813
4 2018-01-05     1475  46.00  46.30  45.75  46.05   52010        2386158


0

### finlab

In [ ]:
# [Finlab Fetch] 抓取 Finlab 處置股資料
from module.get_info_Finlab import FinlabClient

finlab_client = FinlabClient()
print("Fetching disposal information from Finlab...")

# 抓取資料
finlab_disposal = finlab_client.get_data("disposal_information", start_date='2018-01-01')

if not finlab_disposal.empty:
    print(f"Fetched {len(finlab_disposal)} records from Finlab.")
else:
    print("No data fetched from Finlab.")

Fetching disposal information from Finlab...
請從 https://ai.finlab.tw/api_token 複製驗證碼，貼於此處:

輸入成功!
之後可以使用以下方法自動登入
import finlab
finlab.login("YOUR API TOKEN")
Daily usage: 1.1 / 500 MB - disposal_information
Fetched 6757 records from Finlab.
  stock_id       date     證券名稱        處置條件    處置措施  \
0   00642U 2020-03-26  元大S&P石油  監視業務督導會報決議  督導會報決議   
1   00642U 2020-04-06  元大S&P石油  監視業務督導會報決議  督導會報決議   
2   00642U 2020-04-13  元大S&P石油  監視業務督導會報決議  督導會報決議   
3   00642U 2020-04-21  元大S&P石油  監視業務督導會報決議  督導會報決議   
4   00642U 2020-04-28  元大S&P石油  監視業務督導會報決議  督導會報決議   

                                                處置內容     處置開始時間     處置結束時間  \
0  １處置原因：「公布或通知注意交易資訊暨處置作業要點」第六條第四項經「監視業務督導會報」決議。... 2020-03-27 2020-04-13   
1  １處置原因：「公布或通知注意交易資訊暨處置作業要點」第六條第四項經「監視業務督導會報」決議。... 2020-04-07 2020-04-20   
2  １處置原因：「公布或通知注意交易資訊暨處置作業要點」第六條第四項經「監視業務督導會報」決議。... 2020-04-14 2020-04-27   
3  １處置原因：「公布或通知注意交易資訊暨處置作業要點」第六條第四項經「監視業務督導會報」決議。... 2020-04-22 2020-05-06   
4  １處置原因：「公布或通知注意交易資訊暨處置作業要點」第六條第四項經「監視業務督導會

### merge

In [43]:
# --- Event Study Analysis (Using Optimized Module) ---

event_df = run_event_study(price_df, finlab_disposal)

if not event_df.empty:
    display(event_df)
    
    # 寫入 CSV 方便檢查
    event_df.to_csv('test.csv', index=False)
else:
    print("Analysis returned empty DataFrame.")

Running Event Study Analysis...
Detected Finlab data format. Using '處置開始時間' as event start.
Consolidated Start Events: 6757
Analysis completed. Result shape: (18426, 14)


,Date,Stock_id,t_label,relative_day,gap_days,calendar_relative_day,condition,interval,Open,High,Low,Close,Volume,daily_ret
12,2020-03-24,00642U,t-3,-3,0.0,-3,監視業務督導會報決議,5.0,10.32,10.57,10.17,10.36,42796200,0.003876
24,2020-03-25,00642U,t-2,-2,0.0,-2,監視業務督導會報決議,5.0,10.55,10.59,10.44,10.46,39627340,-0.008531
36,2020-03-26,00642U,t-1,-1,0.0,-1,監視業務督導會報決議,5.0,10.44,10.46,10.11,10.33,25842235,-0.010536
48,2020-03-27,00642U,t+0,0,0.0,0,監視業務督導會報決議,5.0,10.24,10.24,10.10,10.17,20479766,-0.006836
60,2020-03-30,00642U,t+1,1,2.0,3,監視業務督導會報決議,5.0,9.88,9.88,9.65,9.79,33984332,-0.009109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260011,2021-07-23,9962,t-1,-1,0.0,-3,連續三次,20.0,24.00,25.60,24.00,25.60,22693000,0.066667
260013,2021-07-26,9962,t+0,0,2.0,0,連續三次,20.0,23.05,24.70,23.05,23.05,7764000,0.000000
260015,2021-07-27,9962,t+1,1,0.0,1,連續三次,20.0,23.80,23.80,22.25,22.50,3173000,-0.054622
260017,2021-07-28,9962,t+2,2,0.0,2,連續三次,20.0,22.50,22.60,21.80,21.80,1769000,-0.031111


# backtest